In [172]:
function old(x::Int)
    print("Old behavior")
end

function new(x::Int)
    print("New behavior")
end

# Adding true/false argument doesn't change my observations.
@deprecate old(x::Int) new(x::Int) false

old (generic function with 2 methods)

In [173]:
old(2)

New behavior

In [203]:
zeros(Float64,1,2,2,2)

1×2×2×2 Array{Float64, 4}:
[:, :, 1, 1] =
 0.0  0.0

[:, :, 2, 1] =
 0.0  0.0

[:, :, 1, 2] =
 0.0  0.0

[:, :, 2, 2] =
 0.0  0.0

In [ ]:
function fill_transition_matrix!(A,qn,λn,w,n)
    shape_A = size(A)
    dimension_A = ndims(A)
    N = shape_A[1]
    incremental_submatrix = zeros(N*2^grid_dimension)
    for grid_dim in 1:dimension_A-1
        B[(multiply(shape_A[1:grid_dim])*(qn[grid_dim]-1)+1) : (multiply(shape_A[1:grid_dim])*(qn[grid_dim]))] .+= log(1-λn[grid_dim])
        for jumps in 1:(multiply(shape_A[grid_dim+1:dimension_A])-1)
            B[(multiply(shape_A[1:grid_dim])*(qn[grid_dim]-1)+1 + multiply(shape_A[1:grid_dim+1])*jumps) : (multiply(shape_A[1:grid_dim])*(qn[grid_dim]) + multiply(shape_A[1:grid_dim+1])*jumps)] .+= log(1-λn[grid_dim])
        end
    end
    B = reshape(exp.(B),shape_A)
    A .+= B
end

In [126]:
function multiply(vector)
    n = length(vector)
    product = 1
    for k in 1:n
        product *= vector[k]
    end
    return product
end

multiply (generic function with 1 method)

In [ ]:
function my_trembling_hand_v3!(A, x, w::Float64) 
    
    shape_A = size(A)
    grid_dimension = ndims(A)-1
    δ = @SVector [1.0./(shape_A[1+i]-1) for i = 1:grid_dimension]


    for n in 1:shape_A[1]

        xn = x[n]
        xn = min.(max.(xn, 0.0),1.0)
        qn = div.(xn, δ)
        qn = max.(0, qn)
        qn = min.(qn, shape_A[2:grid_dimension+1].-2)
        λn = (xn./δ.-qn) # ∈[0,1[ by construction
        qn_ = round.(Int,qn) + 1
        
        
        fill_transition_matrix!(A,qn,λn,w,n)
        
    end

end

In [41]:
function trembling_hand!(A::AbstractArray{Float64,2}, x, w)
    N,n0 = size(A)
    δ0 = 1.0./(n0-1.0)
    for n in 1:N
        x0 = x[n][1]
        x0 = min.(max.(x0, 0.0),1.0)
        q0 = div.(x0, δ0)
        q0 = max.(0, q0)
        q0 = min.(q0, n0-2)
        λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
        q0_ = round.(Int,q0) + 1
        A[n, q0_]   += (1-λ0)*w
        A[n, q0_+1] += λ0*w
    end
end

trembling_hand! (generic function with 1 method)

In [94]:
using StaticArrays
using Dolo
A = Array{Float64,3}(undef, 2, 3, 2)
x = [SVector{2,Float64}(3.4,4.7),SVector{2,Float64}(2.6,4.2)]
w =1.0
my_trembling_hand_v1!(A,x,w)
A

2×3×2 Array{Float64, 3}:
[:, :, 1] =
 5.0e-324  1.5e-323  2.5e-323
 1.0e-323  2.0e-323  3.0e-323

[:, :, 2] =
 3.5e-323  4.4e-323  1.0
 4.0e-323  5.0e-323  1.0

In [85]:
function my_trembling_hand_v1!(A, x, w::Float64) 
    d = length(x[1])
    if d==1
        N,n0 = size(A)
        δ0 = 1.0./(n0-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1
            A[n, q0_]   += (1-λ0)*w
            A[n, q0_+1] += λ0*w
        end
    

    elseif d==2
        N,n0,n1 = size(A)
        δ0 = 1.0./(n0-1.0)
        δ1 = 1.0./(n1-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1

            x1 = x[n][2]
            x1 = min.(max.(x1, 0.0),1.0)
            q1 = div.(x1, δ1)
            q1 = max.(0, q1)
            q1 = min.(q1, n1-2)
            λ1 = (x1./δ1-q1) # ∈[0,1[ by construction
            q1_ = round.(Int,q1) + 1

            A[n, q0_ ,  q1_] += (1-λ0)*(1-λ1)*w
            A[n, q0_+1, q1_] += λ0*(1-λ1)*w
            A[n, q0_, q1_+1] += (1-λ0)*λ1*w
            A[n, q0_+1, q1_+1] += λ0*λ1*w
        end
    
    else 
        print("error : dimension > 2")
    end

end

function my_trembling_hand_v2!(A, x, w::Float64) 
 
    d = length(x[1])
    if d==1
        N,n0 = size(A)
        δ0 = 1.0./(n0-1.0)
        for n in 1:N
            x0 = x[n][1]
            x0 = min.(max.(x0, 0.0),1.0)
            q0 = div.(x0, δ0)
            q0 = max.(0, q0)
            q0 = min.(q0, n0-2)
            λ0 = (x0./δ0-q0) # ∈[0,1[ by construction
            q0_ = round.(Int,q0) + 1
            A[n, q0_]   += (1-λ0)*w
            A[n, q0_+1] += λ0*w
        end
    

    elseif d==2
        N,n0,n1 = size(A)
        vector_n = SVector{2,Float64}(n0,n1)
        δ = SVector{2,Float64}(1.0./(n0-1.0), 1.0./(n1-1.0))
        for n in 1:N

            xn = x[n]
            xn = min.(max.(xn, 0.0),1.0)
            qn = div.(xn, δ)
            qn = max.(0, qn)
            qn = min.(qn, vector_n.-2)
            λn = (xn./δ.-qn) # ∈[0,1[ by construction
            qn_ = round.(Int,qn) + 1

            q0_, q1_ = qn_[1], qn_[2]
            λ0, λ1  = λn[1], λn[2]
            A[n, q0_ ,  q1_] += (1-λ0)*(1-λ1)*w
            A[n, q0_+1, q1_] += λ0*(1-λ1)*w
            A[n, q0_, q1_+1] += (1-λ0)*λ1*w
            A[n, q0_+1, q1_+1] += λ0*λ1*w
        end
    
    else 
        print("error : dimension > 2")
    end

end






my_trembling_hand_v2! (generic function with 1 method)

In [88]:
A = Array{Float64,3}(undef, 4, 3, 2)
exemple = size(A)
exemple[2:3].-2.

(1.0, 0.0)

In [111]:
truc = size(A)


(2, 3, 2)

In [97]:
exemple2 = [2,1]
#print(A)
A[append!([1],exemple2)]
append!([1],exemple2)

3-element Vector{Int64}:
 1
 2
 1

In [98]:
[1,1:3,1:2]
#typeof(1:3)
#UnitRange(1,3)
#UnitRange.(exemple2,exemple2.+1)
vcat([1],UnitRange.(exemple2,exemple2.+1))

3-element Vector{Any}:
 1
  2:3
  1:2

In [100]:
exemple3=[1]
typeof(Tuple(x for x in vcat([1,2],UnitRange.(exemple3,exemple3.+1))))
#A[Tuple(x for x in vcat([1,2],UnitRange.(exemple3,exemple3.+1)))]

Tuple{Int64, Int64, UnitRange{Int64}}

In [108]:
A

2×3×2 Array{Float64, 3}:
[:, :, 1] =
 5.0e-324  1.5e-323  2.5e-323
 1.0e-323  2.0e-323  3.0e-323

[:, :, 2] =
 3.5e-323  4.4e-323  1.0
 4.0e-323  5.0e-323  1.0